# EJERCICIOS SPARK, Ejercicio 4 - Julia Hernández Elena

In [14]:
import os
import pandas as pd

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.window import Window


In [15]:
conf = (

    SparkConf()
    .setAppName(u"[ICAI] Ejercicios Spark")
    .set("spark.executor.memory", "7g")
    .set("spark.executor.cores", "5")
    .set("spark.default.parallelism", 600)
    .set("spark.sql.shuffle.partitions", 600) 
    .set("spark.dynamicAllocation.maxExecutors", 2) 
)

In [3]:
spark = (

    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()

)

# EJERCICIO 4

### 1. Leer los dos dataset a DataFrame de spark:

In [4]:
info_contenidos = spark.read.json("/datos/ejercicio_audis/info_contenidos.json").cache()
type(info_contenidos)

pyspark.sql.dataframe.DataFrame

In [5]:
info_contenidos.show(5)

+--------+------------+
|duracion|id_contenido|
+--------+------------+
|    7014|       93204|
|    9177|      101632|
|     869|      100787|
|    7223|       93047|
|    3600|      101451|
+--------+------------+
only showing top 5 rows



In [6]:
audiencias = spark.read.load('/datos/ejercicio_audis/audiencias.parquet').cache()
type(audiencias)

pyspark.sql.dataframe.DataFrame

In [7]:
audiencias.show(5)

+-------+------------+---------------+---------------------+
|id_user|id_contenido|         franja|segundos_visualizados|
+-------+------------+---------------+---------------------+
| 4810.0|       704.0|   FINDE_MANANA|                 4993|
| 7212.0|       544.0|FINDE_PRIMETIME|                 1204|
| 9794.0|       986.0|FINDE_MADRUGADA|                  130|
|11421.0|       687.0|    FINDE_TARDE|                  123|
|  344.0|      4776.0|    ENTRE_NOCHE|                 3391|
+-------+------------+---------------+---------------------+
only showing top 5 rows



### 2. ¿Cuántos registros tienen el dataset audiencias?
### ¿Cuántos registros tienen el dataset info_contenidos?

In [8]:
audiencias.count() #registros de audiencias

25595651

In [9]:
info_contenidos.count() #registros de info_contenidos

116290

### 3. ¿Cuántos usuarios distintos hay?
### ¿Cuántos contenidos distintos han sido reproducidos al menos una vez?

In [10]:
users_distintos = (

    audiencias.select(
        F.countDistinct('id_user').alias('Usuarios distintos'),
        F.countDistinct('id_contenido').alias('Contenidos distintos')
        #*al reproducirse al menos una vez va a aparecer en el fichero
    )
).show()

+------------------+--------------------+
|Usuarios distintos|Contenidos distintos|
+------------------+--------------------+
|             43926|               11629|
+------------------+--------------------+



### 4. ¿Cuántos contenidos distintos han sido reproducidos al más de 5 veces?

In [11]:
contenido_masde_5 = (

    audiencias
    .groupBy('id_contenido')
    .count()
    .filter('count>=5')
    .count()
    
)

In [12]:
print("Contenidos distintos vistos mas de cinco veces: {}" .format(contenido_masde_5))

Contenidos distintos vistos mas de cinco veces: 10431


### 5. Es posible que un usario haya reproducido más de una vez el mismo contenidos, se quiere llegar a un nuevo DF que lo llamaremos `ratings` dónde solo aparezca una vez cada pareja de usuario/contenido. En este DF se guardará la `franja` con más segundos visualizados de ese contenido y la suma total de segundos visualizado.

In [16]:
ratings = (
    
    audiencias
    
    #Sumamos segundos para cada usuario contenido y franja
    .groupBy('id_user', 'id_contenido', 'franja')
    .agg( F.sum('segundos_visualizados').alias('segundos_visualizados'))
    
    #Calculamos la franja con mas segundos de visualizacion
    .withColumn(
        'dummy', 
        F.row_number()
        .over(
            Window
            .partitionBy("id_user","id_contenido")
            .orderBy(F.desc('segundos_visualizados'))
        )
    )
    
    #Sumamos los segundos totales 
    .withColumn(
        'segundos_totales', 
        F.sum('segundos_visualizados')
        .over(
            Window
            .partitionBy('id_user', 'id_contenido')
        )
    )
    
    #Nos quedamos con la franja con mas segundos
    .filter('dummy=1')
    
    
    #Seleccionamos las variables
    .select('id_contenido', 'id_user', 'franja', 'segundos_totales')
    
    
).cache()
    

In [17]:
#Chequeamos que lo hemos hecho correctamente filtrando por el usuario y el contenido que venian en el enunciado
audiencias.filter('id_user=357.0').filter('id_contenido=1131.0').show()
ratings.filter('id_user=357.0').filter('id_contenido=1131.0').show()

+-------+------------+---------------+---------------------+
|id_user|id_contenido|         franja|segundos_visualizados|
+-------+------------+---------------+---------------------+
|  357.0|      1131.0|ENTRE_PRIMETIME|                  596|
|  357.0|      1131.0|ENTRE_PRIMETIME|                  302|
|  357.0|      1131.0|ENTRE_PRIMETIME|                  644|
|  357.0|      1131.0|ENTRE_PRIMETIME|                  383|
|  357.0|      1131.0|FINDE_SOBREMESA|                   96|
+-------+------------+---------------+---------------------+

+------------+-------+---------------+----------------+
|id_contenido|id_user|         franja|segundos_totales|
+------------+-------+---------------+----------------+
|      1131.0|  357.0|ENTRE_PRIMETIME|            2021|
+------------+-------+---------------+----------------+



### 6. Una vez calculado el DF ratings, cruza con la duración total del contenido para cada pareja usuario/contenidos, y calcular la siguiente columna nueva: 
### rating = min(segundos_totales/duracion, 1)

In [21]:
ratings = (
    
    ratings

    #Hago un Join con la duracion del contenido
    .join(info_contenidos, 'id_contenido')
    
    #Calculamos el porcentaje de segundos vistos
    .withColumn('ratings', F.col('segundos_totales') / F.col('duracion') )
    
    #Le damos un valor de 1 si el % es mayor que 1
    .withColumn('rating', 
               F.when(F.col('ratings')>1, 1).otherwise(F.col('ratings')) )
    
    #Seleccionamos los campos que nos interesan
    .select('id_contenido', 'id_user', 'franja', 'rating')
    
).cache()

In [22]:
#Chequeamos que lo hemos hecho correctamente filtrando por el usuario y el contenido que venian en el enunciado
ratings.filter('id_user=357.0').filter('id_contenido=1131.0').show()

+------------+-------+---------------+-------------------+
|id_contenido|id_user|         franja|             rating|
+------------+-------+---------------+-------------------+
|      1131.0|  357.0|ENTRE_PRIMETIME|0.34874892148403797|
+------------+-------+---------------+-------------------+



### 7. Analizar con describe el valor de esta nueva variable rating:

In [25]:
#Haceomos el describe de rating redondeando los valores a 4 decimales para facilitar la interpretacion
ratings.describe('rating').withColumn('rating', F.round('rating', 4)).show()

+-------+-----------+
|summary|     rating|
+-------+-----------+
|  count|1.5342422E7|
|   mean|     0.4024|
| stddev|     0.3807|
|    min|     1.0E-4|
|    max|        1.0|
+-------+-----------+



### 8. Guardar el DF ratings con el mismo nombre en hive, en el esquema personal de cada uno:

In [105]:
mi_user = os.environ.get('USER') #cogemos nuetro nombre de usuario

In [106]:
mi_user

'jhelena'

In [108]:
table_name = mi_user + ".ratings" #nombramos la tabla

In [109]:
ratings.write.mode('overwrite').saveAsTable(table_name)

In [26]:
spark.stop()